# CBS

In [ ]:
# all imports
from instaloader import Post
import pandas as pd
import os
import instaloader
import sys
import re
import numpy as np

filepath = '../data/'
filename = 'Coosto_berichten2.csv'

In [40]:
# functions    
def outputCSV(dataset, filename):
    dataset.to_csv(filepath + filename, sep=';')
    
def cprint(text):
    sys.stdout.write("\r" + text)
    sys.stdout.flush()

In [41]:
# retrieve data
insta = pd.read_csv(filepath + filename, delimiter=';')

# remove empty columns
insta = insta.drop(['zoekopdracht', 'type', 'volgers', 'invloed', 'titel', 'type bron'], axis=1)

# check with count()
insta.count()

datum               22
url                 22
sentiment            6
discussielengte     19
views                5
auteur              22
GPS breedtegraad     2
GPS lengtegraad      2
bericht tekst       22
dtype: int64

In [42]:
# Download all posts from instagram using an array of urls
def get_posts(urls):
    posts_dict = {}
    total_length = len(urls)
    
    for index, url in enumerate(urls):
        shortcode = url.split("/")[-2]
        
        try:
            L = instaloader.Instaloader()
            post = Post.from_shortcode(L.context, url.split("/")[-2])
            posts_dict[shortcode] = post
        except:
            pass
        
        cprint("Getting posts " + str(round((index / total_length) * 100)) + "% completed")
        
    return posts_dict


# Get the indexes of the posts which do not exist anymore
def get_non_exsisting_posts(dataset, posts_dict):
    indexes_to_drop = []
    
    for index, row in dataset.iterrows():
        shortcode = row['url'].split("/")[-2]
        if not shortcode in posts_dict:
            indexes_to_drop.append(index)
    
    return indexes_to_drop
            

# Delete posts from the dataset based on an array of indexes
def del_posts(data, indexes_to_drop):
    for index in indexes_to_drop:
        data = data.drop(index=index, axis=0)
    return data


# Enrich dataset with like count
def add_like_count_to_dataset(dataset, posts_dict):
    for index, row in dataset.iterrows():
        shortcode = row['url'].split("/")[-2]
        if shortcode in posts_dict:
            dataset.at[index, 'likes count'] = posts_dict[shortcode].likes
    return dataset


# Adds utc date to the dataset
def add_date_utc(dataset, posts_dict):
    for index, row in dataset.iterrows():
        shortcode = row['url'].split('/')[-2]
        if shortcode in posts_dict:
            dataset.at[index, 'datum utc'] = posts_dict[shortcode].date_utc
            
    return dataset


# Refreshes comment count
def refresh_comment_count(dataset, posts_dict):
    for index, row in dataset.iterrows():
        shortcode = row['url'].split('/')[-2]
        if shortcode in posts_dict:
            dataset.at[index, 'discussielengte'] = posts_dict[shortcode].comments
            
    return dataset


# Refreshes view count
def refresh_views(dataset, posts_dict):
    for index, row in dataset.iterrows():
        shortcode = row['url'].split('/')[-2]
        if shortcode in posts_dict:
            dataset.at[index, 'views'] = posts_dict[shortcode].likes
            
    return dataset

# Cleans invalid urls and enriches with like count, data utc, comment count and view count
def clean_und_enrich(dataset):
    posts_dict = get_posts(dataset['url'])
    indexes_to_drop = get_non_exsisting_posts(dataset, posts_dict)
    dataset = del_posts(dataset, indexes_to_drop)
    dataset = add_like_count_to_dataset(dataset, posts_dict)
    dataset = add_date_utc(dataset, posts_dict)
    dataset = refresh_comment_count(dataset, posts_dict)
    dataset = refresh_views(dataset, posts_dict)
    
    cprint('\nInvalid urls found: ' + str(len(indexes_to_drop)))
    
    return dataset

def improve_sentiment(dataset):
    dataset['sentiment'] = dataset['sentiment'].replace(np.nan, '0')
    
    return dataset

def isolate_hashtag(data):  
    total_hashtags = []
    for index, row in data.iterrows():
        text = row['bericht tekst'] 
        print(text)
        
        # find all hashtags in text and isolate them in new column
        total_hashtags.append(re.findall(r"#(\w+)", text))
        
        #remove hashtags from text
        pattern = re.compile("#(\w+)")
        newText = pattern.sub(r'', text)
        data.at[index, 'bericht tekst'] = newText
        
    data['hashtags'] = total_hashtags
    return data

def remove_emoji(data):  
    indexes_to_drop = []
        
    for index, row in data.iterrows():
        a = row['bericht tekst']

        # todo: vul aan met meer emoji's
        emoji_pattern = re.compile("["
                          u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
        newValue = emoji_pattern.sub(r'', a)
        newValue = newValue.replace('🥗', '')
        if newValue == '' : 
            indexes_to_drop.append(index)
            
        else:
            data.at[index, 'bericht tekst'] = newValue
    
    pass
    data = del_posts(data, indexes_to_drop)
    return data

In [43]:
insta = improve_sentiment(insta)
insta = remove_emoji(insta)
insta = clean_und_enrich(insta)
insta = isolate_hashtag(insta)

# Resets index
insta.index = range(len(insta))

Getting posts 95% completed
Invalid urls found: 2@_korobo_  кстати хочу сказать, что коробочки снимать гораздо сложней.  Я пробовала свои отснять. Не просто, так сказать....
So chic
Incredible
@divaribas.82 aloe y fibern plus. Escribame 0994895191
@remkevet oooh wat goed haha, heerlijk!:)
Lovely Japanese copy of Pin Ups. One of my favourites - soon after dark Emily cry's. #vinyljunkie #vinyladdict #welltempered #davidbowie #nowspinning
@amanda.carlberg
@nobbsy91  look at this!
     .......#
I can't wait to spend this weekend at #wondercon - - #tbt to Wondercon last year when I ran into @brucewayne626's amazing Armored Batman.
Schönen Abend für Euch 
Het programma word steeds slechter!!! Het nieuwe format is al niet om aan te zien, nu ook nog zo dramatische patatbakker als coach? Zonde van het programma #tvoh #notdone
Fancy salad  #ski #selfie #luxe #glam #bridal #bossbabe #follow #skier #like #followers #fit #fitgirl #fitnessmodel #fitness #fitspo #fitlife #fitnessaddict #fitnessjourne

In [44]:
insta.head(20)

,datum,url,sentiment,discussielengte,views,auteur,GPS breedtegraad,GPS lengtegraad,bericht tekst,likes count,datum utc,hashtags
0,2017-06-20 17:06,https://instagram.com/p/BVjjp3Cl4uK/,0,8.0,31.0,marusinalavka,NaN,NaN,"@_korobo_ кстати хочу сказать, что коробочки ...",31.0,2017-06-20 09:00:29,[]
1,2017-10-10 21:43,https://instagram.com/p/BaFFd0fBbcw/,0,62.0,3639.0,artofobservance,NaN,NaN,So chic,3639.0,2017-10-10 19:36:27,[]
2,2017-04-28 16:47,https://instagram.com/p/BTbF7AEDgHg/,0,1370.0,534004.0,amigabali,NaN,NaN,Incredible,534004.0,2017-04-28 09:03:58,[]
3,2017-09-04 22:22,https://instagram.com/p/BYoVU-GFypi/,0,43.0,2224.0,gpazminoyepez,NaN,NaN,@divaribas.82 aloe y fibern plus. Escribame 09...,2224.0,2017-09-04 19:05:41,[]
4,2017-11-08 10:29,https://instagram.com/p/BbOnSBlFwmo/,+,12.0,46.0,laurienblomphotography,NaN,NaN,"@remkevet oooh wat goed haha, heerlijk!:)",46.0,2017-11-08 08:56:30,[]
5,2017-07-20 08:02,https://instagram.com/p/BWwfFxkFoHW/,0,0.0,63.0,breaking_glass,NaN,NaN,Lovely Japanese copy of Pin Ups. One of my fav...,63.0,2017-07-20 06:02:08,"[vinyljunkie, vinyladdict, welltempered, david..."
6,2018-02-10 22:27,https://instagram.com/p/Bdp4B0Pj6TL/,0,9411.0,223473.0,belmacurkicc,NaN,NaN,@amanda.carlberg,223473.0,2018-01-07 16:05:07,[]
7,2018-02-21 18:01,https://instagram.com/p/BfdzxFYlj-h/,+,699.0,103847.0,_lilysmith97_,NaN,NaN,@nobbsy91 look at this!,103847.0,2018-02-21 16:39:47,[]
8,2017-08-28 17:40,https://instagram.com/p/BYVy0NRBGdr/,0,2.0,17.0,heimint,NaN,NaN,.......#,17.0,2017-08-28 14:17:47,[]
9,2017-03-30 22:45,https://instagram.com/p/BSRrHmbDSui/,0,9.0,91.0,bsunphotography,34.041531,-118.269028,I can't wait to spend this weekend at - - to...,91.0,2017-03-30 20:45:12,"[wondercon, tbt]"


In [31]:
# output new cleaned dataset
data = insta
outputCSV(data, "cleaned.csv")

In [5]:
chunksize = 2500
made_files = 0

def split_file(made_files):
    for i,chunk in enumerate(pd.read_csv(filepath + filename, delimiter=';', chunksize=chunksize)):
        chunk = chunk.drop(['zoekopdracht', 'type', 'volgers', 'invloed', 'titel', 'type bron'], axis=1)
        chunk.to_csv(filepath + 'set{}.csv'.format(i), sep=';')
        made_files +=1
    return made_files

made_files = split_file(made_files)
print(made_files)
    

In [5]:
#Cleaning al data files after another
def clean_splitted_files():
    for num in range(0, made_files):
        setfile = pd.read_csv(filepath + "set"+ str(num) + '.csv', delimiter=';')

        setfile = improve_sentiment(setfile)
        setfile = remove_emoji(setfile)
        setfile = clean_und_enrich(setfile)
        setfile = isolate_hashtag(setfile)

        # Resets index
        outputCSV(setfile,"set"+ str(num) + '.csv')

clean_splitted_files()

In [5]:
#Mergin all all datafile in one file
def merge_splitted_files_to_one:
    merged_filename = "merged_file.csv"
    try:
        os.remove(filepath + merged_filename)
    except OSError:
        pass

    outputfile = open(filepath + merged_filename, "a", encoding="utf8")
    for line in open(filepath + "set0.csv", encoding="utf8"):
        outputfile.write(line)
    # now the rest:    
    for num in range(1,made_files):
        setfile = open(filepath +"set"+ str(num) +".csv", encoding="utf8")
        setfile.__next__() # skip the header
        for line in setfile:
             outputfile.write(line)
        setfile.close() # not really needed
    outputfile.close()